# IMPORTS

In [1]:
from arcgishub import hub
from arcgis.gis import GIS

import configparser
import json
import os


# VARIABLES

In [4]:
# initiative_id = "5a9bc8dfb3e54817ac61fa4d8aa33cc0"  # PROD
initiative_id = "5d230c46f10b4c91a60c54e9bca879b6"  # J's demo site cloned to mdimapdatacatalog account

# Credentials access and variable creation
credentials_file = fr"..\Credentials\Credentials.cfg"

assert os.path.exists(credentials_file)
config_parser = configparser.ConfigParser()
config_parser.read(credentials_file)

# md_url = config_parser["DEFAULT"]["url_hub"]
md_url = config_parser["DEFAULT"]["url_maps"]
md_admin = config_parser["DEFAULT"]["login"]
md_pwd = config_parser["DEFAULT"]["password"]


# FUNCTIONALITY

In [7]:
# Check on the creds being used
# print(f"URL: {md_url}")
# print(f"ADMIN: {md_admin}")
# print(f"PWD: {md_pwd}")

In [ ]:
# Create a gis connection with regular arcgis api
gis = GIS(url=md_url, username=md_admin, password=md_pwd) # "old" way
my_hub = gis.hub
prod_initiative = my_hub.initiatives.get(initiative_id)
prod_initiative.site_url

In [ ]:
# Create a connection to the hub using the "new" arcgishub module
my_hub = hub.Hub(url=md_url, username=md_admin, password=md_pwd)
print(my_hub.enterprise_org_id)

In [ ]:
# Searches for all initiatives visible to the account being used - takes a little time to run
# try:
#     print(my_hub.initiatives.search())
# except json.JSONDecodeError as jde:
#     print(f"y_hub.initiatives() returned {jde}")

In [ ]:
# Gets a specific initiative
try:
    print(my_hub.initiatives.get(initiative_id))
except json.JSONDecodeError as jde:
    print(f"y_hub.initiatives() returned {jde}")